In [1]:
import numpy as np
import pandas as pd
import string
import re

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

In [34]:
def remove_punch(text):
    x = str(text)
    for p in string.punctuation:
        x = x.replace(p,'')
    return x

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def lower_tokens(tokens):
    return  [w.lower() for w in tokens]

def remove_stopwords(tokens):
    stop = set(stopwords.words("english"))
    filtered_words = [word for word in tokens if word not in stop]
    return " ".join(filtered_words)

def remove_names(text):
    for word in text.split():
        if word[0] == "@":
            text = text.replace(word, "")
    return text

def remove_url(text):
    result = re.sub(r"http\S+", "", text)
    result = re.sub(r"https\S+", "", text)
    return result


def decode_unicode(text):
    text = text.decode('utf-8')
    return text


def text_to_wordlist(text,stem_words=True):
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"doesn't", "does not ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # remove consecutive letters to single letter at end
    text = re.sub(r'(.)\1+$', r'\1', text)
    
    # Return a list of words
    return(text)

In [17]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
df = pd.read_csv('en_Hasoc2021_train.csv')

In [19]:
df['text_clean'] = df['text'].apply(lambda text:remove_names(text))
df['text_clean'] = df['text_clean'].apply(lambda text:remove_url(text))
df['text_clean'] = df['text_clean'].apply(lambda text:text_to_wordlist(text))

In [20]:
df['text_clean'] = df['text_clean'].apply(lambda text:deEmojify(text))
df['text_clean'] = df['text_clean'].apply(lambda text:remove_punch(text))

In [21]:
tokens = [word_tokenize(tweet) for tweet in df['text_clean']]
lower = [lower_tokens(token) for token in tokens]

In [33]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [39]:
removed = []
for tokens in lower:
    removed.append(remove_stopwords(tokens))

In [41]:
df['text_clean'] = pd.Series(removed)

In [42]:
df.head()

,Unnamed: 0,_id,text,task_1,task_2,text_clean
0,4986,60c5d6bf5659ea5e55defa2c,@wealth if you made it through this &amp;&amp;...,HOF,PRFN,made amp amp onli abl start make money sustain...
1,3394,60c5d6bf5659ea5e55def461,Technically that's still turning back the cloc...,HOF,OFFN,technic still turn back clock dick head
2,1310,60c5d6bf5659ea5e55defaad,@VMBJP @BJP4Bengal @BJP4India @narendramodi @J...,NOT,NONE,govt stop think world media liber gang ani opt...
3,3390,60c5d6bf5659ea5e55def419,@krtoprak_yigit Soldier of Japan Who has dick ...,HOF,OFFN,soldier japan dick head
4,4626,60c5d6bf5659ea5e55def7fa,@blueheartedly You'd be better off asking who ...,HOF,OFFN,would better ask think sleazi shitbag lmao


In [43]:
df.to_csv('preprocess_data.csv')